# Titanic Kaggle Coding Club

1. train.csv:
train.csv는 탑승한 승객들 정보를 .csv파일 형태로 가지고 있음. <br>
(엑셀 파일 형태로 정리되어 있다고 생각) <br>
총 891명의 승객 정보를 가지고 있음
- Survied column이 target column -> 우리가 예측해야하는 특징값 (feature)
    - "1"이면 살아 남은 것, "0"이면 죽은 것

2. test.csv:
test.csv 역시 탑승한 승객들 정보를 .csv 파일 형태로 가지고 있음. <br>
(엑셀 파일 형태로 정리되어 있다고 생각) <br>
총 418명의 승객 정보를 가지고 있음.
- test.csv에는 "Survived" column이 없음
    - 가려져 있고 나는 알 수 없음 -> 검사할 때 알려줌

3. gender_submission.csv:
예측 csv파일을 어떻게 만들어야 하는지 예시를 보여줌
- submission file에는 "PassengerId"랑 "Survived" column이 있어야함


### Step 1 : Data download
1. `pip install kaggle` -> 오류 발생 시 `pip install --user kaggle`

2. API Key 발급

API credentials
To use the Kaggle API, sign up for a Kaggle account at https://www.kaggle.com. Then go to the 'Account' tab of your user profile (https://www.kaggle.com/<username>/account) and select 'Create API Token'. This will trigger the download of kaggle.json, a file containing your API credentials. Place this file in the location ~/.kaggle/kaggle.json (on Windows in the location C:\Users\<Windows-username>\.kaggle\kaggle.json - you can check the exact location, sans drive, with echo %HOMEPATH%). You can define a shell environment variable KAGGLE_CONFIG_DIR to change this location to $KAGGLE_CONFIG_DIR/kaggle.json (on Windows it will be %KAGGLE_CONFIG_DIR%\kaggle.json).

For your security, ensure that other users of your computer do not have read access to your credentials. On Unix-based systems you can do this with the following command:

chmod 600 ~/.kaggle/kaggle.json

You can also choose to export your Kaggle username and token to the environment:

export KAGGLE_USERNAME=datadinosaur
export KAGGLE_KEY=xxxxxxxxxxxxxx
In addition, you can export any other configuration value that normally would be in the $HOME/.kaggle/kaggle.json in the format 'KAGGLE_' (note uppercase).
For example, if the file had the variable "proxy" you would export KAGGLE_PROXY and it would be discovered by the client.

- 혹은 아래 블로그 참고
https://polarcompass.tistory.com/147

설치 완료 후 아래 명령어 수행<br>
`kaggle competitions download -c titanic`

### import packages

In [3]:
import numpy as np # 선형 대수를 위한 package
import pandas as pd # 데이터 처리를 위한 package -> .csv파일 읽고 쓰기
import os

# data 파일에 train, test 데이터가 있는지 확인
for dirname, _, filenames in os.walk('./data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./data/test.csv
./data/train.csv
./data/gender_submission.csv


### load dataset

In [4]:
train_data = pd.read_csv("./data/train.csv") # load train data
train_data.head() # csv파일의 위에 5개 요소 출력 
# 엑셀파일에서 맨 위에 5개 요소 출력해준다고 생각 -> 프린트 된거 보면 엑셀 같쥬?

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
# 동일한 방법으로 test dataset 로드
test_data = pd.read_csv("./data/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### 우리의 목표
- train.csv에서 패턴을 찾아서, test.csv에 있는 승객들이 살았는지 죽었는지 예측 하는 것

- gender_submission.csv를 보고 패턴을 파악해보자
    - gender_submission.csv는 모든 여성 승객들이 살고, 남성 승객들이 죽었다고 가정(예측)하였다.<br> 
        -> 이 예측이 합리적인가 확인해봅시다 == 모든 여성이 살고 남성이 죽었다는 패턴이 합리적인지 확인해보자
- how?
    - train.csv의 패턴을 파악해서, 정말 여성 승객들이 얼마나 살았는지 그 패턴을 한번 봐보자 이 말임.

In [12]:
# train_data에서 여성 (Sex=='female') 인 사람들만 추출하고 그 중 "Survived" column값만 가져옴
women = train_data.loc[train_data.Sex == 'female']["Survived"]
# 살아남았으면 1이고, 아니면 0이니까 survived column값을 다 더하면 살아남은 여성들의 수를 구할 수 있음 -> sum(women)을 쓰는 이유
# len(women)은 해당 dataframe의 길이를 반환 -> women만 추출된 데이터의 길이
rate_women = sum(women)/len(women)

# 살아남은 여성의 비율을 출력
print("% of women who survived: ", rate_women)

% of women who survived:  0.7420382165605095


In [13]:
# 위와 같은 방법으로 살아남은 남성의 비율을 출력 

men = train_data.loc[train_data.Sex=='male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived: ", rate_men)

% of men who survived:  0.18890814558058924


### 패턴 분석
- 약 75프로의 여성이 살아남았고, 19프로의 남성이 살아남음.
- 꽤 괜찮은 경향성이지만 아직 많이 부족<br>
    -> 이렇게 되면 입력 데이터가 여성이면 15% 오류 발생 가능성, 남성이면 18% 오류 발생 가능성 존재
- gender_based submission은 오직 `Sex`라는 하나의 feature column (특징값)만 고려해서 살았는지 죽었는지 판단 -> 다른 요소들은 깡그리 무시

- 여러개의 column(승객의 특징)들을 결합하여 더 복잡한 패턴을 발견할 수 있고, 이것은 더 좋은 예측으로 이끈다.

### Random forest model을 사용해보자

- 마치 아키네이터처럼 승객이 죽었는지 살았는지 판단
- 예를들면:
    - question 1: 승객이 남자인가 여자인가
    - question 2-1: 남자라면 1등석에 탔는가 2등석에 탔는가 3등석에 탔는가
    - question 3-1-1: 남자인데 1등석에 탔다면 동승자가 1명인가 2명인가 ...
    - ...(이어지는 질문)
    - 남자고, 1등석에 탔고, 동승자가 2명이고, ... , 는 대체로 죽었다. 라는 패턴 파악 <br>
        -> 이를 근거로 판단
- sklearn.ensemble에서 RandomForestClassifier 모델을 import 해서 사용

In [14]:
# package import
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"] # 죽었는지 살았는지 정답만 추출 = y

# 4개의 특성 (객실 등급, 성별, 형제 or 배우자 수, 부모 or 자녀 수)만 패턴 파악에 고려
features = ["Pclass", "Sex", "SibSp", "Parch"] 

# pd.get_dummies: 모든 데이터를 수치로 변환해주는 전처리 작업
# 예를들어 월, 화, 수, 목, 금 과 같이 표현되어 있는 데이터를 (0, 1, 2, 3, ...)으로 나타내줌
# 사실 0, 1, 2로 표현 안하고 (0, 1, 2에 이미 수치적 의미가 있기 때문 -> 화요일이 월요일보다 큰 수를 가질 이유가 없잖아)
# 월 = (1, 0, 0, ...), 화 = (0, 1, 0, ...), 수 = (0, 0, 1, ...)로 표현
X = pd.get_dummies(train_data[features]) # train_data

X_test = pd.get_dummies(test_data[features]) # test_data: test data로 train data와 같은 전처리를 가짐

#모델 객체 생성
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

#.fit()함수는 그냥 학습하는 거라고 생각하면 됌
# 더 정확한 알고리즘은 2학년 or 3학년때 배워용
model.fit(X, y)

# .fit을 통해 훈련된 model을 가지고 predict
predictions = model.predict(X_test)

# 제출할 수 있도록 "PassengerId" column과 "Survived" column만 남긴 Dataframe을 만듬
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
# 이걸 .csv 파일로 생성 -> 사이트에 제출하려고
output.to_csv('submission.csv', index=False)

print("Your submission was successfully saved!")

Your submission was successfully saved!


### Submit

- -c option: competition name
- -f option: submission csv file name
- -m option: submission description (설명글)

In [15]:
!kaggle competitions submit -c titanic -f submission.csv -m "tutorial"

100%|████████████████████████████████████████| 2.77k/2.77k [00:03<00:00, 801B/s]
Successfully submitted to Titanic - Machine Learning from Disaster

### view results

In [16]:
!kaggle competitions submissions -c titanic

fileName        date                 description  status    publicScore  privateScore  
--------------  -------------------  -----------  --------  -----------  ------------  
submission.csv  2023-05-05 07:41:21  tutorial     complete  0.77511                    
